<!--
authors: Matthew Wilson, Daniele Reda
created: 2020/01/14
last_updated: 2023/02/08
-->


## CPSC 533V: Assignment 3 - Tabular Q Learning and DQN (Due Thu Feb 22)

---

#  Part 1 [54 pts] Tabular Q-Learning 

Tabular Q-learning is an RL algorithm for problems with discrete states and discrete actions. The algorithm is described in the class notes, which borrows the summary description from [Section 6.5](http://incompleteideas.net/book/RLbook2018.pdf#page=153) of Richard Sutton's RL book. In the tabular approach, the Q-value is represented as a lookup table. As discussed in class, Q-learning can further be extended to continuous states and discrete actions, leading to the [Atari DQN](https://arxiv.org/abs/1312.5602) / Deep Q-learning algorithm.  However, it is important and informative to first fully understand tabular Q-learning.

Informally, Q-learning works as follows: The goal is to learn the optimal Q-function: 
`Q(s,a)`, which is the *value* of being at state `s` and taking action `a`.  Q tells you how well you expect to do, on average, from here on out, given that you act optimally.  Once the Q function is learned, choosing an optimal action is as simple as looping over all possible actions and choosing the one with the highest Q (optimal action $a^* = \text{argmax}_a Q(s,a)$).  To learn Q, we initialize it arbitrarily and then iteratively refine it using the Bellman backup equation for Q functions, namely: 
$Q(s,a) \leftarrow Q(s,a) + \alpha [r + \gamma \text{max}_a Q(s', a) - Q(s,a)]$.
Here, $r$ is the reward associated with with the transition from state s to s', and $\alpha$ is a learning rate.

In the first part of assignment you will implement tabular Q-learning and apply it to CartPole -- an environment with a **continuous** state space.  To apply the tabular method, you will need to discretize the CartPole state space by dividing the state-space into bins.


**Goals:**
- to become familiar with python/numpy, as well as using an OpenAI Gym environment
- to understand tabular Q-learning, by implementing tabular Q-Learning for 
  a discretized version of a continuous-state environment, and experimenting with the implementation
- (optional) to develop further intuition regarding possible variations of the algorithm

## Introduction
Deep reinforcement learning has generated impressive results for board games ([Go][go], [Chess/Shogi][chess]), video games ([Atari][atari], [DOTA2][dota], [StarCraft II][scii]), [and][baoding] [robotic][rubix] [control][anymal] ([of][cassie] [course][mimic] ;)).  RL is beginning to work for an increasing range of tasks and capabilities.  At the same time, there are many [gaping holes][irpan] and [difficulties][amid] in applying these methods. Understanding deep RL is important if you wish to have a good grasp of the modern landscape of control methods.

These next several assignments are designed to get you started with deep reinforcement learning, to give you a more close and personal understanding of the methods, and to provide you with a good starting point from which you can branch out into topics of interest. You will implement basic versions of some of the important fundamental algorithms in this space, including Q-learning and policy gradient/search methods.

We will only have time to cover a subset of methods and ideas in this space.
If you want to dig deeper, we suggest following the links given on the course webpage.  Additionally we draw special attention to the [Sutton book](http://incompleteideas.net/book/RLbook2018.pdf) for RL fundamentals and in depth coverage, and OpenAI's [Spinning Up resources](https://spinningup.openai.com/en/latest/spinningup/rl_intro.html) for a concise intro to RL and deep RL concepts, as well as good comparisons and implementations of modern deep RL algorithms.


[atari]: https://arxiv.org/abs/1312.5602
[go]: https://deepmind.com/research/case-studies/alphago-the-story-so-far
[chess]:https://deepmind.com/blog/article/alphazero-shedding-new-light-grand-games-chess-shogi-and-go 
[dota]: https://openai.com/blog/openai-five/
[scii]: https://deepmind.com/blog/article/AlphaStar-Grandmaster-level-in-StarCraft-II-using-multi-agent-reinforcement-learning
[baoding]: https://bair.berkeley.edu/blog/2019/09/30/deep-dynamics/
[rubix]: https://openai.com/blog/solving-rubiks-cube/
[cassie]: https://www.cs.ubc.ca/~van/papers/2019-CORL-cassie/index.html
[mimic]: https://www.cs.ubc.ca/~van/papers/2018-TOG-deepMimic/index.html
[anymal]: https://arxiv.org/abs/1901.08652


[irpan]: https://www.alexirpan.com/2018/02/14/rl-hard.html
[amid]: http://amid.fish/reproducing-deep-rl



In [2]:
# # uncomment if necesary
# !pip install numpy
# !pip install gym
# # OR:
# !pip install gymnasium
import time
import itertools
import numpy as np
# import gym
import gymnasium as gym

---

## [12 pts] Explore the CartPole environment 

Your first task is to familiarize yourself with the OpenAI gym interface and the [CartPole environment](https://github.com/Farama-Foundation/Gymnasium/blob/main/gymnasium/envs/classic_control/cartpole.py)
by writing a simple hand-coded policy to try to solve it.  
To begin understanding OpenAI Gym environments, [read this first](https://gymnasium.farama.org/api/env/).) 
The gym interface is very popular and you will see many algorithm implementations and 
custom environments that support it.  You may even want to use the API in your course projects, 
to define a custom environment for a task you want to solve.

Note that there were several breaking changes introduced in the past few years to the gym API. Some reference algorithm implementations online might be using the old version:
- `obs = env.reset()` ->  `obs, info = env.reset()`
- `obs, reward, done, info = env.step(action)` to `obs, reward, terminated, truncated, info = env.step(action)`
- `env.render()` no longer accepts the `render_mode` parameter (e.g. human mode where the environment is rendered in a pop-up window, or rgb_array which allows headless conversion to images or videos)


Below is some example code that runs a simple random policy.  You are to:
- **run the code to see what it does**
- **write code that chooses an action based on the observation**.  You will need to learn about the gym API and to read the CartPole documentation to figure out what the `action` and `obs` vectors mean for this environment. 
Your hand-coded policy can be arbitrary, and it should ideally do better than the random policy.  There is no single correct answer. The goal is to become familiar with `env`s.
- **write code to print out the total reward gained by your policy in a single episode run**
- **answer the short-response questions below** (see the TODOs for all of this)

In [3]:
env = gym.make('CartPole-v1', render_mode="human")  # you can also try LunarLander-v2, but make sure to change it back
print('observation space:', env.observation_space)
print('action space:', env.action_space)

# To find out what the observations mean, read the CartPole documentation.
# Uncomment the lines below, or visit the source file: 
# https://github.com/Farama-Foundation/Gymnasium/blob/main/gymnasium/envs/classic_control/cartpole.py

#cartpole = env.unwrapped
#cartpole?

observation space: Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
action space: Discrete(2)


In [4]:
def policy(obs):
    # if angle in dangerous position, move according to angle
    if obs[2] < -0.18 or obs[2] > 0.18:
        return int(obs[2] > 0)

    # if not near edges
    if obs[0] < 2.1 and obs[0] > -2.1:
        # only based on angular speed
        return int(obs[3] > 0)

    # turn left if near right boundary
    if obs[0] >= 2.1:
        return 0

    # turn right if near left boundary
    return 1

In [5]:
# 1.1 [10pts]


for policy_mode in ['manual', 'random']:
    total_reward = []
    for i in range(10):
        env = gym.make('CartPole-v1') # (render_mode='human') does not work on M2 mac
        # runs a single episode and render it.  try running this before editing anything
        obs, info = env.reset()  # get first obs/state
        rewards_all = []
        while True:
            # TODO: replace this `action` with something that depends on `obs`
            # action = env.action_space.sample()  # random action
            action = policy(obs) if policy_mode == 'manual' else env.action_space.sample()
            obs, reward, terminated, truncated, info = env.step(action)
            rewards_all.append(reward)
            env.render()
            time.sleep(0.1)  # so it doesn't render too quickly
            if terminated | truncated: break
        env.close()

        # TODO: print out your total sum of rewards here
        print (f'Reward Sum for {i}th iteration and policy {policy_mode}: {np.sum(rewards_all)}')
        total_reward.append(np.sum(rewards_all))

    print (f'For policy mode: {policy_mode}, maximum reward: {np.max(total_reward)}, average reward: {np.mean(total_reward)}')

/Users/nikhilshenoy/miniforge3/envs/rl/lib/python3.9/site-packages/gymnasium/envs/classic_control/cartpole.py:215: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("CartPole-v1", render_mode="rgb_array")
  gym.logger.warn(


Reward Sum for 0th iteration and policy manual: 132.0
Reward Sum for 1th iteration and policy manual: 128.0
Reward Sum for 2th iteration and policy manual: 167.0
Reward Sum for 3th iteration and policy manual: 190.0
Reward Sum for 4th iteration and policy manual: 137.0
Reward Sum for 5th iteration and policy manual: 195.0
Reward Sum for 6th iteration and policy manual: 166.0
Reward Sum for 7th iteration and policy manual: 139.0
Reward Sum for 8th iteration and policy manual: 155.0
Reward Sum for 9th iteration and policy manual: 140.0
For policy mode: manual, maximum reward: 195.0, average reward: 154.9
Reward Sum for 0th iteration and policy random: 11.0
Reward Sum for 1th iteration and policy random: 10.0
Reward Sum for 2th iteration and policy random: 21.0
Reward Sum for 3th iteration and policy random: 14.0
Reward Sum for 4th iteration and policy random: 27.0
Reward Sum for 5th iteration and policy random: 35.0
Reward Sum for 6th iteration and policy random: 18.0
Reward Sum for 7th 

To answer the questions below, look at the full [source code here](https://github.com/Farama-Foundation/Gymnasium/blob/main/gymnasium/envs/classic_control/cartpole.py) if you haven't already.

**1.2. [2pts] Briefly describe your policy.  What observation information does it use?  What score did you achieve (rough maximum and average)?  And how does it compare to the random policy?**

My policy is based on the following code where I use the x position, angle and angular speed to determine my action,
```python
def policy(obs):
    # if angle in dangerous position, move according to angle
    if obs[2] < -0.18 or obs[2] > 0.18:
        return int(obs[2] > 0)

    # if not near edges
    if obs[0] < 2.1 and obs[0] > -2.1:
        # only based on angular speed
        return int(obs[3] > 0)
    
    # turn left if near right boundary
    if obs[0] >= 2.1:
        return 0
    
    # turn right if near left boundary
    return 1
```
I ran both the random and my policy for 10 iterations to get the following statistics,
- Random Policy: maximum reward: 35.0, average reward: 18.2
- My policy: maximum reward: 195.0, average reward: 154.9

---

##  [12 pts] Discretize the env

Next, we need to discretize CartPole's continuous state space to work for tabular Q-learning.  While this is in part  a contrived usage of tabular methods, given the existence of other approaches that are designed to cope with continuous state-spaces, it is also interesting to consider whether tabular methods can be adapted more directly via discretization of the state into bins. Furthermore, tabular methods are simple, interpretabile, and can be proved to converge, and thus they still remain relevant.

Your task is to discretize the state/observation space so that it is compatible with tabular Q-learning.  To do this:
- **implement `obs_normalizer` to pass its test**
- **implement `get_bins` to pass its test**
- **then answer question 2.3**

[map]: https://arxiv.org/abs/1504.04909
[qd]: https://quality-diversity.github.io/

In [9]:
env = gym.make('CartPole-v1')

In [115]:
# 2.1 [5 pts for passing test_normed]
def obs_normalizer(obs):
    """Normalize the observations between 0 and 1
    
    If the observation has extremely large bounds, then clip to a reasonable range before normalizing; 
    (-2,2) should work.  (It is ok if the solution is specific to CartPole)
    
    Args:
        obs (np.ndarray): shape (4,) containing an observation from CartPole using the bound of the env
    Returns:
        normed (np.ndarray): shape (4,) where all elements are roughly uniformly mapped to the range [0, 1]
    
    """
    # HINT: check out env.observation_space.high, env.observation_space.low
    lo, hi = env.observation_space.low, env.observation_space.high
    
    # for large bounds clip the low, hi to a max_value
    max_value = 5
    lo = np.clip(lo, -max_value, max_value)
    hi = np.clip(hi, -max_value, max_value)
    obs = np.clip(obs, -max_value, max_value)

    return (obs - lo) / (hi - lo)
    # # TODO: implement this function
    # raise NotImplementedError('TODO')

In [116]:
### TEST 2.1
def test_normed():
    obs, info = env.reset()
    while True:
        obs, _, terminated, truncated, _ =  env.step(env.action_space.sample())
        normed = obs_normalizer(obs) 
        assert np.all(normed >= 0.0) and np.all(normed <= 1.0), '{} are outside of (0,1)'.format(normed)
        if terminated | truncated:
            break
    env.close()
    print('Passed!')
test_normed()

Passed!


In [118]:
# 2.2 [5 pts for passing test_binned]
def get_bins(normed, num_bins):
    """Map normalized observations (0,1) to bin index values (0,num_bins-1)
    
    Args:
        normed (np.ndarray): shape (4,) output from obs_normalizer
        num_bins (int): how many bins to use
    Returns:
        binned (np.ndarray of type np.int32): shape (4,) where all elements are values in range [0,num_bins-1]
    
    """
    # TODO: implement this function
    bins = np.arange(0, 1.0 + (1/num_bins), (1/num_bins))
    return np.clip(np.digitize(normed, bins) - 1, 0, num_bins-1).astype(np.int32)
    # raise NotImplementedError('TODO')

In [119]:
### TEST 2.2
obs, info = env.reset()

def test_binned(num_bins):
    normed = np.array([0.0, 0.2, 0.8, 1.0])
    binned = get_bins(normed, num_bins)
    assert np.all(binned >= 0) and np.all(binned < num_bins), '{} supposed to be between (0, {})'.format(binned, num_bins-1)
    assert binned.dtype == np.int32, "You should also make sure to cast your answer to int using arr.astype(np.int32)" 
    
test_binned(5)
test_binned(10)
test_binned(50)
print('Passed!')

Passed!


**2.3. [2 pts] If your state has 4 values and each is binned into N possible bins, how many bins are needed to represent all unique possible states?**


Every state has N possible bins, so the total number of unique possible states is $4N$.

---

## [20 pts] Solve the env 

Using the pseudocode below and the functions you implemented above, implement tabular Q-learning and use it to solve CartPole.

We provide setup code to initialize the Q-table and give examples of interfacing with it. Write the inner and outer loops to train your algorithm.  These training loops will be similar to those deep RL approaches, so get used to writing them!

The algorithm (excerpted from Section 6.5 of [Sutton's book](http://incompleteideas.net/book/RLbook2018.pdf)) is given below:

![Sutton RL](https://i.imgur.com/mdcWVRL.png)

in summary:
- **implement Q-learning using this pseudocode and the helper code**
- **answer the questions below**
- **run the suggested experiments and otherwise experiment with whatever interests you**

In [120]:
# setup (see last few lines for how to use the Q-table)
env = gym.make('CartPole-v1')

# hyper parameters. feel free to change these as desired and experiment with different values
num_bins = 10
alpha = 0.1
gamma = 0.99
log_n = 1000
# epsilon greedy
eps = 0.05  #usage: action = optimal if np.random.rand() > eps else random

obs, info = env.reset()

# Q-table initialized to zeros.  first 4 dims are state, last dim is for action (0,1) for left,right.
Q = np.zeros([num_bins]*len(obs)+[env.action_space.n])

# helper function to convert observation into a binned state so we can index into our Q-table
obs2bin = lambda obs: tuple(get_bins(obs_normalizer(obs), num_bins=num_bins))

s = obs2bin(obs)

print('Shape of Q Table: ', Q.shape) # you can imagine why tabular learning does not scale very well
print('Original obs {} --> binned {}'.format(obs, s))
print('Value of Q Table at that obs/state value', Q[s])

Shape of Q Table:  (10, 10, 10, 10, 2)
Original obs [-0.01000857  0.00094133  0.03893396  0.04370571] --> binned (4, 5, 5, 5)
Value of Q Table at that obs/state value [0. 0.]


In [147]:
# 3.1 [20 pts]

# TODO: implement Q learning, following the pseudo-code above. 
#     - you can follow it almost exactly, but translating things for the gym api and our code used above
#     - make sure to use e-greedy, where e = random about 0.05 percent of the time
#     - make sure to do the S <-- S' step because it can be easy to forget
#     - every log_n steps, you should render your environment and
#       print out the average total episode rewards of the past log_n runs to monitor how your agent trains
#      (your implementation should be able to break at least +150 average reward value, and you can use that 
#       as a breaking condition.  It make take several minutes to run depending on your computer.)


def run_one_episode(Q, alpha: float, eps: float, gamma: float, obs2bin: callable):
    '''Run one episode using Tabular Q-learning'''
    episode_reward = 0
    env = gym.make('CartPole-v1') # (render_mode='human') does not work on M2 mac
    obs, _ = env.reset()  # get first obs/state
    s = obs2bin(obs)
    while True:        
        # select action based on epsilon-greedy
        action = np.argmax(Q[s]) if np.random.rand() > eps else np.random.randint(0, 2)
        obs, reward, terminated, truncated, _ = env.step(action)
        episode_reward += reward

        # new state
        new_s = obs2bin(obs)

        # update Q function based on equation
        Q[s][action] += alpha * (reward + gamma * np.max(Q[new_s]) - Q[s][action])

        # break if terminated or truncated
        if terminated | truncated:
            Q[s] = np.zeros(2) # for terminal state Q value should be 0
            break
        
        # update state with new observation
        s = new_s

    env.close()
    return episode_reward

def experiment_setup(num_bins, alpha, gamma, log_n, eps):
    env = gym.make('CartPole-v1')
    Q = np.zeros([num_bins]*len(obs)+[env.action_space.n])

    print (f'For alpha: {alpha}, eps: {eps}, gamma: {gamma}')

    # define obs2bin
    obs2bin = lambda obs: tuple(get_bins(obs_normalizer(obs), num_bins=num_bins))

    total_rewards = []
    total_episodes = 0
    while True:
        reward = run_one_episode(Q, alpha, eps, gamma, obs2bin)
        total_rewards.append(reward)
        total_episodes += 1
        
        # compute avg_reward for last log_n runs
        avg_reward = np.mean(total_rewards[-log_n:])
        
        # breaking condition if it crosses 150    
        if avg_reward > 150:
            break

        if total_episodes > 20_000:
            print (f'Did not converge. At 20_000 episodes, average return in the last {log_n} episodes: {avg_reward}')
            return total_episodes
        
    print (f'Ran for a total of {total_episodes} episodes before getting 150 as average reward!')
    return total_episodes

# hyper parameters. feel free to change these as desired and experiment with different values
num_bins = 10
alpha = 0.1
gamma = 0.99
log_n = 1000
eps = 0.05

experiment_setup(num_bins, alpha, gamma, log_n, eps)

For alpha: 0.1, eps: 0.05, gamma: 0.99
Ran for a total of 5303 episodes before getting 150 as average reward!


5303

## [10 pts] Experiments

Given a working algorithm, you will run a few experiments.  Either make a copy of your code above to modify, or make the modifications in a way that they can be commented out or switched between (with boolean flag if statements).

**4.2. [5 pts] $\epsilon$-greedy.**  How sensitive are the results to the value of $\epsilon$?   First, write down your prediction of what would happen if $\epsilon$ is set to various values, including for example [0, 0.05, 0.25, 0.5].

The results are very sensitive to the value of $\epsilon$, as we need to explore more intially and explore less in the later stages when we have a good Q value estimate. For the given values, my prediction would be that for,
- For $\epsilon = 0$, we would be stuck on all states where a non-optimal action has a higher Q value assigned. Therefore, never converging to good Q values.
- For $\epsilon = 0.05$, will take longer to find good Q value estimates since we are exploring only 5% of our actions. This is helpful in the later parts when we have good Q estimates since we pick optimal actions most of the time.
- For $\epsilon = 0.25$, this is useful to get good Q value estimates sooner as we explore 25% of the time, but in the later stages not beneficial as we want to pick the optimal action majority of the time.
- For $\epsilon = 0.5$, since we pick optimal actions 50% of the time, this might lead to delayed convergence of the Q value estimates. Even after convergence, we will not be getting good episodic rewards, since we are picking non-optimal actions with 50% probability.

Now run the experiment and observe the impact on the algorithm.  Report the results below.

In [134]:
num_bins = 10
alpha = 0.1
gamma = 0.99
log_n = 1000
for eps in [0.0, 0.05, 0.25, 0.5]:
    experiment_setup(num_bins, alpha, gamma, log_n, eps)

For alpha: 0.1, eps: 0.0, gamma: 0.99
Did not converge. At 20_000 episodes, average return in the last 1000 episodes: 9.322
For alpha: 0.1, eps: 0.05, gamma: 0.99
Ran for a total of 3157 episodes before getting 150 as average reward!
For alpha: 0.1, eps: 0.25, gamma: 0.99
Ran for a total of 7119 episodes before getting 150 as average reward!
For alpha: 0.1, eps: 0.5, gamma: 0.99
Did not converge. At 20_000 episodes, average return in the last 1000 episodes: 81.04


''

##### Analysis:
The algorithm does not lead to good average reward in $20_000$ episodes for $\epsilon = 0.0$ and $\epsilon = 0.5$. This is because of either exploring too much ($\epsilon = 0.5$) or exploiting non-optimal actions from the beginning ($\epsilon = 0.0$). For $\epsilon = 0.25$, we get the average reward after later than $\epsilon = 0.05$, because for $\epsilon = 0.25$ we still explore when we have found good Q value estimates.

**4.3. [5 pts] Design your own experiment.** Design a modification that you think would either increase or reduce performance.  A simple example (which you can use) is initializing the Q-table differently, and thinking about how this might alter performance. Write down your idea, what you think might happen, and why.

My modification would be to have modify the epsilon greedy parameter such that it is high value in the beginning and it is gradually reduced over the course of training. This allows us to converge quickly to good Q values while allowing us to select mostly optimal value once we have good Q values. In order to do this, I propose,
```python
gamma_epsilon = 0.9
epsilon = 0.2
T_interval = 100

# after every T_interval episodes
if episode_count % T_interval == 0:
    epsilon *= gamma_epsilon
```

Run the experiment and report the results.

In [149]:
def modified_experiment_setup(num_bins, alpha, gamma, log_n, eps, gamma_eps, T_interval):
    env = gym.make('CartPole-v1')
    Q = np.zeros([num_bins]*len(obs)+[env.action_space.n])

    print (f'For alpha: {alpha}, eps: {eps}, gamma: {gamma}')

    # define obs2bin
    obs2bin = lambda obs: tuple(get_bins(obs_normalizer(obs), num_bins=num_bins))

    total_rewards = []
    total_episodes = 0
    while True:
        reward = run_one_episode(Q, alpha, eps, gamma, obs2bin)
        total_rewards.append(reward)
        total_episodes += 1
        
        # Modification
        if total_episodes % T_interval == 0:
            eps *= gamma_eps # reduce epsilon by a bit
        
        # compute avg_reward for last log_n runs
        avg_reward = np.mean(total_rewards[-log_n:])
        
        # breaking condition if it crosses 150    
        if avg_reward > 150:
            break

        if total_episodes > 20_000:
            print (f'Did not converge. At 20_000 episodes, average return in the last {log_n} episodes: {avg_reward}')
            return total_episodes
        
    print (f'Ran for a total of {total_episodes} episodes before getting 150 as average reward!')
    return total_episodes

In [160]:
num_bins = 10
alpha = 0.1
gamma = 0.99
log_n = 1000

eps = 0.3
T_interval = 100
gamma_eps = 0.9

modified_experiment_setup(num_bins, alpha, gamma, log_n, eps=eps, gamma_eps=gamma_eps, T_interval=T_interval)

For alpha: 0.1, eps: 0.3, gamma: 0.99
Ran for a total of 1643 episodes before getting 150 as average reward!


1643

TODO: answer here

---

## A. Extensions (fully optional, will not be graded, if you have time after Part 2)

- plots your learning curve, using e.g., matplotlib 
- visualize the Q-table to see which values are being updated and not
- design a better binning strategy that uses fewer bins for a better-performing policy
- extend this approach to work on different environments (e.g., LunarLander-v2)
- extend this approach to work on environments with continuous actions, by using a fixed set of discrete samples of the action space.  e.g., for Pendulum-v0
- implement a simple deep learning version of this.  we will see next part that DQN uses some tricks to make the neural network training more stable.  Experiment directly with simply replacing the Q-table with a Q-Network and train the Q-Network using gradient descent with `loss = (targets - Q(s,a))**2`, where `targets = stop_grad(R + gamma * maxa(Q(s,a))`).

# Part 2 [60 pts] Behavioral Cloning and Deep Q Learning

---
The second part of assignment will help you transition from tabular approaches to deep neural network approaches. You will implement the [Atari DQN / Deep Q-Learning](https://arxiv.org/abs/1312.5602) algorithm, which arguably kicked off the modern Deep Reinforcement Learning craze.

In this part we will use PyTorch as our deep learning framework.  To familiarize yourself with PyTorch, your first task is to use a behavior cloning (BC) approach to learn a policy.  Behavior cloning is a supervised learning method in which there exists a dataset of expert demonstrations (state-action pairs) and the goal is to learn a policy $\pi$ that mimics this expert.  At any given state, your policy should choose the same action the export would.

Since BC avoids the need to collect data from the policy you are trying to learn, it is relatively simple. 
This makes it a nice stepping stone for implementing DQN. Furthermore, BC is relevant to modern approaches---for example its use as an initialization for systems like [AlphaGo][go] and [AlphaStar][star], which then use RL to further adapte the BC result.  

<!--

I feel like this might be better suited to going lower in the document:

Unfortunately, in many tasks it is impossible to collect good expert demonstrations, making

it's not always possible to have good expert demonstrations for a task in an environemnt and this is where reinforcement learning comes handy. Through the reward signal retrieved by interacting with the environment, the agent learns by itself what is a good policy and can learn to outperform the experts.

-->

Goals:
- Famliarize yourself with PyTorch and its API including models, datasets, dataloaders
- Implement a supervised learning approach (behavioral cloning) to learn a policy.
- Implement the DQN objective and learn a policy through environment interaction.

[go]:  https://deepmind.com/research/case-studies/alphago-the-story-so-far
[star]: https://deepmind.com/blog/article/alphastar-mastering-real-time-strategy-game-starcraft-ii

## Submission information

- Complete by editing and executing the associated Python files.
- Copy and paste the code and the terminal output requested in the predefined cells on this Jupyter notebook.
- When done, upload the completed Jupyter notebook (ipynb file) on canvas.

## Preliminaries

### PyTorch

If you have never used PyTorch before, we recommend you follow this [60 Minutes Blitz][blitz] tutorial from the official website. It should give you enough context to be able to complete the assignment.


**If you have issues, post questions to Piazza**

### Installation

To install all required python packages:

```
python3 -m pip install -r requirements.txt
```

### Debugging


You can include:  `import ipdb; ipdb.set_trace()` in your code and it will drop you to that point in the code, where you can interact with variables and test out expressions.  We recommend this as an effective method to debug the algorithms.


[blitz]: https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html

## 1. [36 pts] Behavioral Cloning

Behavioral Cloning is a type of supervised learning in which you are given a dataset of expert demonstrations tuple $(s, a)$ and the goal is to learn a policy function $\hat a = \pi(s)$, such that $\hat a = a$.

The optimization objective is $\min_\theta D(\pi(s), a)$ where $\theta$ are the parameters the policy $\pi$, in our case the weights of a neural network, and where $D$ represents some difference between the actions.

---

Before starting, we suggest reading through the provided files.

For Behavioral Cloning, the important files to understand are: `model.py`, `dataset.py` and `bc.py`.

- The file `model.py` has the skeleton for the model (which you will have to complete in the following questions),

- The file `dataset.py` has the skeleton for the dataset the model is being trained with,

- and, `bc.py` will have all the structure for training the model with the dataset.


### [10 pts] 1.1 Dataset

We provide a pickle file with pre-collected expert demonstrations on CartPole from which to learn the policy $\pi$. The data has been collected from an expert policy on the environment, with the addition of a small amount of gaussian noise to the actions.

The pickle file contains a list of tuples of states and actions in `numpy` in the following way:

```
[(state s, action a), (state s, action a), (state s, action a), ...]
```

In the `dataset.py` file, we provide skeleton code for creating a custom dataset. The provided code shows how to load the file.

Your goal is to overwrite the `__getitem__` function in order to return a dictionary of tensors of the correct type.

Hint: Look in the `bc.py` file to understand how the dataset is used.

Answer the following questions:

**[6 pts]** Insert your code in the placeholder below.

In [1]:
# PLACEHOLDER TO INSERT YOUR __getitem__ method here
import torch

def __getitem__(self, index):
    item = self.data[index]
    return {'state': torch.from_numpy(item[0]), 'action': torch.tensor([item[1]], dtype=torch.int32)}

**[2 pt]** How big is the dataset provided?

**Answer**: The dataset consists of 99660 state-action pairs.

**[2 pts]** What is the dimensionality of $s$ and what range does each dimension of $s$ span?  I.e., how much of the state space does the expert data cover? What are the dimensionalities and ranges of the action $a$ in the dataset (how much of the action space does the expert data cover)?

**Answer**: The dimensionality of s is 4. For each dimension, these are the following ranges the expert data covers,
- Dimension 0: $[-0.722, 2.399]$
- Dimension 1: $[-0.433, 1.846]$
- Dimension 2: $[-0.050, 0.146]$
- Dimension 3: $[-0.381, 0.471]$

The dimensionality of the actions space is 1 and the range of the action is $[0, 1]$. There are 48555 actions with value 0 and 51105 actions with value 1.


### [5 pts] 1.2 Environment

Recall the state and action space of CartPole, from the previous assignment.

Considering the full state and action spaces, do you think the provided expert dataset has good coverage?  Why or why not? How might this impact the performance of our cloned policy?

**Answer**: The (allowed) observation state space of CartPole has the following ranges,
$$
\text{Min}: [-2.4, -\infty, -0.201, -\infty]\\
\text{Max}: [ 2.4,  \infty,  0.201,  \infty]
$$
whereas the expert data does not have good coverage as it does not cover the full allowed observable state space. For instance, if position of the cart is $< -0.722$, there is no expert data for the policy to learn from. This might impact the cloned policy as certain parts of the state-space might be out-of-distribution for it.

### [14 pts] 1.3 Model

The file `model.py` provides skeleton code for the model. Your goal is to create the architecture of the network by adding layers that map the input to output.

You will need to update the `__init__` method and the `forward` method.

The `select_action` method has already been written for you.  This should be used when running the policy in the environment, while the `forward` function should be used at training time.

- **[10 pts]** Insert your code in the placeholder below.

In [5]:
# PLACEHOLDER TO INSERT YOUR MyModel class here
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self, state_size: int = 4, action_size: int = 1):
        super().__init__()
        self.state_size = state_size
        self.action_size = action_size

        self.net = nn.Sequential(
            nn.Linear(self.state_size, 32),
            nn.SiLU(),
            nn.Linear(32, 32),
            nn.SiLU(),
            nn.Linear(32, 32),
            nn.SiLU(),
            nn.Linear(32, 32),
            nn.SiLU(),           
            nn.Linear(32, self.action_size)
        )
        # TODO YOUR CODE HERE FOR INITIALIZING THE MODEL
        # Guidelines for network size: start with 2 hidden layers and maximum 32 neurons per layer
        # feel free to explore different sizes

    def forward(self, x):
        # TODO YOUR CODE HERE FOR THE FORWARD PASS
        return self.net(x) 
    
    def select_action(self, state):
        self.eval()
        x = self(state)
        self.train()
        return x.max(1)[1].view(1, 1).to(torch.long)


Answer the following questions:

- **[2 pts]** What is the dimension and meaning of the input of the network

**Answer**: The dimension of the input of the network is 4 since the state space of CartPole is 4-dimensional. The input will be (batch_size, 4), where batch_size is the number of state-action pairs used in the mini-batch.

- **[2 pts]** Similarly, describe the output.

**Answer**: The dimension of the output of the network is 2 since the actions space is 1-dimensional with 2 possible actions. The output values will be un-normalized logits for the two actions with the shape (batch_size, 2).


### [7 pts] 1.4 Training

The file `bc.py` is the entry point for training your behavioral cloning model. The skeleton and the main components are already there.

The missing parts for you to do are:

- Initializing the model
- Choosing a loss function
- Choosing an optimizer
- Playing with hyperparameters to train your model.

- **[5 pts]** Insert your code in the placeholder below.

In [1]:
# PLACEHOLDER FOR YOUR CODE HER
# HOW DID YOU INITIALIZE YOUR MODEL, OPTIMIZER AND LOSS FUNCTIONS? PASTE HERE YOUR FINAL CODE
# NOTE: YOU CAN KEEP THE FOLLOWING LINES COMMENTED OUT, AS RUNNING THIS CELL WILL PROBABLY RESULT IN ERRORS

# model = MyModel(state_size=env.observation_space.shape[0], action_size=env.action_space.n)
# optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)
# loss_function = torch.nn.CrossEntropyLoss()

You can run your code by doing:

```
python3 bc.py
```

**During all of this assignment, the code in `eval_policy.py` will be your best friend.** At any time, you can test your model by giving as argument the path to the model weights and the environment name using the following command:

```
python3 eval_policy.py --model-path /path/to/model/weights --env ENV_NAME
````

In [4]:
# ## PASTE YOUR TERMINAL OUTPUT HERE
# # NOTE: TO HAVE LESS LINES PRINTED, YOU CAN SET THE VARIABLE PRINT_INTERVAL TO 5 or 10
# [epoch    1/10] [iter       0] [loss 0.69425]
# [epoch    2/10] [iter    2000] [loss 0.00878]
# [Test on environment] [epoch 2/10] [score 248.50]
# [epoch    3/10] [iter    4000] [loss 0.01058]
# [epoch    4/10] [iter    6000] [loss 0.02082]
# [Test on environment] [epoch 4/10] [score 242.60]
# [epoch    6/10] [iter    8000] [loss 0.00495]
# [Test on environment] [epoch 6/10] [score 275.50]
# [epoch    7/10] [iter   10000] [loss 0.01813]
# [epoch    8/10] [iter   12000] [loss 0.00271]
# [Test on environment] [epoch 8/10] [score 273.50]
# [epoch    9/10] [iter   14000] [loss 0.00324]
# [Test on environment] [epoch 10/10] [score 250.30]
# Saving model as behavioral_cloning_CartPole-v1.pt

**[2 pts]** Did you manage to learn a good policy? How consistent is the reward you are getting?

**Answer**: Yes, the policy consistently achieves more than 250 reward in the first 10 epochs.

## 2. [24 pts] Deep Q Learning

There are two main issues with the behavior cloning approach.

- First, we are not always lucky enough to have access to a dataset of expert demonstrations.
- Second, replicating an expert policy suffers from compounding error. The policy $\pi$ only sees these "perfect" examples and has no knowledge on how to recover from states not visited by the expert. For this reason, as soon as it is presented with a state that is off the expert trajectory, it will perform poorly and will continue to deviate from a good trajectory without the possibility of recovering from errors.

---
The second task consists in solving the environment from scratch, using RL, and most specifically the DQN algorithm, to learn a policy $\pi$.

For this task, familiarize yourself with the file `dqn.py`. We are going to re-use the file `model.py` for the model you created in the previous task.

Your task is very similar to the one in the previous assignment, to implement the Q-learning algorithm, but in this version, our Q-function is approximated with a neural network.

The algorithm (excerpted from [Atari DQN paper](https://arxiv.org/abs/1312.5602)) is given below:

![DQN algorithm](https://i.imgur.com/Mh4Uxta.png)

### 2.0 [2 pts] Think about your model...



In DQN, we are using the same model as in task 1 for behavioral cloning. In both tasks the model receives as input the state and in both tasks the model outputs something that has the same dimensionality as the number of actions. These two outputs, though, represent very different things. What is each one representing?

**Answer**: In task 1, the output of the neural network represented the action probabilities/logits since we were cloning policies. Here, the Q function represents the Q-value for each action, given the state.

### 2.1 [10 pts] Update your Q-function

Complete the `optimize_model` function. This function receives as input a `state`, an `action`, the `next_state`, the `reward` and `done` representing the tuple $(s_t, a_t, s_{t+1}, r_t, done_t)$. Your task is to update your Q-function as shown in the [Atari DQN paper](https://arxiv.org/abs/1312.5602) environment. For now don't be concerned with the experience replay buffer. We'll get to that later.

![Loss function](https://i.imgur.com/tpTsV8m.png)

Insert your code in the placeholder below.

In [8]:
## PLACEHOLDER TO INSERT YOUR optimize_model function here:

# def optimize_model(state, action, next_state, reward, done):
#     # TODO given a tuple (s_t, a_t, s_{t+1}, r_t, done_t) update your model weights
#     reward = torch.tensor(reward, device=device).float()
#     next_state = torch.from_numpy(next_state).float()
#     state = torch.from_numpy(state).float()
#     loss = F.mse_loss(
#         reward + (1 - done) * GAMMA * torch.max(target(next_state)), # target
#         model(state)[action].squeeze() # prediction
#     )

#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()

### 2.2 [5 pts] $\epsilon$-greedy strategy

You will need a strategy to explore your environment. The standard strategy is to use $\epsilon$-greedy. Implement it in the `choose_action` function template.

Insert your code in the placeholder below.

In [7]:
## PLACEHOLDER TO INSERT YOUR choose_action function here:

# def choose_action(state, test_mode=False):
#     state = torch.from_numpy(state).float()
#     return (
#         torch.argmax(model(state), keepdim=True)
#         if (random.uniform(0, 1) > EPS_EXPLORATION or test_mode)
#         else torch.randint(0, n_actions, (1,), dtype=torch.long, device=device)
#     ).view(-1, 1)

### 2.3 [2 pts] Train your model

Try to train a model in this way.

You can run your code by doing:

```
python3 dqn.py
```

How many episodes does it take to learn (ie. reach a good reward)?

**YOUR ANSWER HERE**

In [9]:
## PASTE YOUR TERMINAL OUTPUT HERE
# NOTE: TO HAVE LESS LINES PRINTED, YOU CAN SET THE VARIABLE PRINT_INTERVAL TO 5 or 10
# Have attached the output in the file 2.2.txt

### 2.4 [5 pts] Add the Experience Replay Buffer

If you read the DQN paper (and as you can see from the algorithm picture above), the authors make use of an experience replay buffer to learn faster. We provide an implementation in the file `replay_buffer.py`. Update the `train_reinforcement_learning` code to push a tuple to the replay buffer and to sample a batch for the `optimize_model` function.

In [ ]:
## PASTE YOUR TERMINAL OUTPUT HERE
# NOTE: TO HAVE LESS LINES PRINTED, YOU CAN SET THE VARIABLE PRINT_INTERVAL TO 5 or 10


How does the replay buffer improve performance?

**YOUR ANSWER HERE**

## 3. Extra (fully optional)

Ideas to experiment with:

- Is $\epsilon$-greedy strategy the best strategy available? Experiment with other strategies.
- Make use of the model you have trained in the behavioral cloning part and fine-tune it with RL. How does that affect performance?
- You are perhaps bored with `CartPole-v1` by now. Another environment we suggest trying is `LunarLander-v2`. It will be harder to learn but with experimentation, you will find the correct optimizations for success. Piazza is also your friend :)
- What about learning from images? This requires more work because you have to extract the image from the environment. How much more challenging might you expect the learning to be in this case?
- An improvement over DQN is DoubleDQN. Experiment with this to see how much of an impact it makes.



In [ ]:
# YOU CAN USE THIS CODEBLOCK AND ADD ANY BLOCK BELOW AS YOU NEED
# TO SHOW US THE IDEAS AND EXTRA EXPERIMENTS YOU RUN.
# HAVE FUN!